# Convert ROOT files Barfeature

In [1]:
distances = [15]
doubleplanes = [30]
energies = [600]
erels = [500]
neutrons = [1, 2, 3, 4, 5, 6]
physicss = ["inclxx"]
subruns = range(20)

In [2]:
import h5py
import numpy as np
import os
import sys
import joblib
import ROOT

sys.path.append("..")
from helpers import filename_for

ROOT.ROOT.EnableThreadSafety()

Welcome to JupyROOT 6.16/00


To create the HDF5 file, it is good to know the number of events to store ahead of time. With ROOT, this requires a bit more code than it should, as you can't have multiple files opened.

In [3]:
def root2hdf5(distance, doubleplane, energy, erel, neutron, physics, subrun):
    inpfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "digi.root")
    outfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "bars.h5")

    with h5py.File(outfile, "w") as h5file:
        tfile = ROOT.TFile.Open(os.fspath(inpfile))
        ttree = tfile.Get("evt")

        num_events = ttree.GetEntries()
        num_bars = doubleplane * 100
        chunksize = 1000

        ## trifeature
        dim = 6
        trifeature = h5file.create_dataset(
            "trifeature",
            shape=(num_events, dim),
            dtype=np.int16,
            chunks=(chunksize, dim),
            compression="gzip",
            compression_opts=9,
        )
        tbuff = np.zeros((dim), np.int16)

        ## flatfeatures
        flatfeatures = h5file.create_dataset(
            "flatfeatures",
            shape=(num_events, num_bars * 2),
            dtype=np.float32,
            chunks=(chunksize, num_bars * 2),
            compression="gzip",
            compression_opts=9,
        )
        fbuff = np.zeros((num_bars * 2), np.float32)

        ## primhitsbars
        primhitsbars = h5file.create_dataset(
            "primhitsbars",
            shape=(num_events, num_bars),
            dtype=np.int8,
            chunks=(chunksize, num_bars),
            compression="gzip",
            compression_opts=9,
        )
        pbuff = np.zeros((num_bars), np.int8)

        i = 0
        for event in ttree:
            ## trifeature
            # nPN: Number of incoming primary neutrons
            tbuff[0] = event.NeulandPrimaryTracks.GetEntries()
            # nPP: Number of primary neutrons with an energy deposition in NeuLAND
            tbuff[1] = event.NeulandPrimaryPoints.GetEntries()
            # nPH: Number of hits that correspond to a energy deposition of a primary neutron
            tbuff[2] = event.NeulandPrimaryHits.GetEntries()
            # nHits: Number of hits
            tbuff[3] = event.NeulandHits.GetEntries()
            # nClus: Number of clusters
            tbuff[4] = event.NeulandClusters.GetEntries()
            # Edep: Total deposited (detected) energy
            tbuff[5] = round(sum([hit.GetE() for hit in event.NeulandHits]))
            trifeature[i] = tbuff

            ## flatfeatures
            fbuff.fill(0)
            for hit in event.NeulandHits:
                bar = hit.GetPaddle() - 1
                fbuff[2 * bar + 0] = hit.GetE()
                fbuff[2 * bar + 1] = hit.GetT()
            flatfeatures[i] = fbuff

            ## primhitsbars
            pbuff.fill(0)
            for phit in event.NeulandPrimaryHits:
                bar = phit.GetPaddle() - 1
                pbuff[bar] = 1
            primhitsbars[i] = pbuff

            i += 1

        tfile.Close()

In [4]:
joblib.Parallel(n_jobs=-1, backend="multiprocessing")(
    joblib.delayed(root2hdf5)(
        distance=distance,
        doubleplane=doubleplane,
        energy=energy,
        erel=erel,
        neutron=neutron,
        physics=physics,
        subrun=subrun,
    )
    for distance in distances
    for energy in energies
    for doubleplane in doubleplanes
    for neutron in neutrons
    for erel in erels
    for physics in physicss
    for subrun in subruns
)

print("Done")

Done
